In [1]:
import pandas as pd
import numpy as np
import sqlite3
import re

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## Loading data / processing

In [2]:
conn_metro = sqlite3.connect('./police_london.db')

In [ ]:
query_metro_street = 'SELECT * FROM METRO_STREET'
df_metro_street = pd.read_sql(query_metro_street, conn_metro)

In [ ]:
query_metro_outcomes = 'SELECT * FROM METRO_OUTCOMES'
df_metro_outcomes = pd.read_sql(query_metro_outcomes, conn_metro)

In [ ]:
query_metro_search = 'SELECT * FROM METRO_SEARCH'
df_metro_search = pd.read_sql(query_metro_search, conn_metro)

In [ ]:
# mdp uses 12 bcu in the frontline, all covering a certain no. of boroughs (possibly needed for later analysis)

boroughs_bcu = {'Westminster': 'AW - Central West', 'Kensington and Chelsea': 'AW - Central West',
                'Hammersmith and Fulham': 'AW - Central West', 'Wandsworth': 'SW - South West',
                'Lambeth': 'AS - Central South', 'Southwark': 'AS - Central South',
                'Tower Hamlets': 'CE - Central East', 'Hackney': 'CE - Central East',
                'Islington': 'CN - Central North', 'Camden': 'CN - Central North',
                'Brent': 'NW - North West', 'Ealing': 'WA - West Area', 
                'Hounslow': 'WA - West Area', 'Richmond upon Thames': 'SW - South West', 
                'Kingston upon Thames': 'SW - South West', 'Merton': 'SW - South West', 
                'Sutton': 'SN - South Area', 'Croydon': 'SN - South Area', 'Bromley': 'SN - South Area', 
                'Lewisham': 'SE - South East', 'Greenwich': 'SE - South East', 
                'Bexley': 'SE - South East', 'Havering': 'EA - East Area', 
                'Barking and Dagenham': 'EA - East Area', 'Redbridge': 'EA - East Area',
                'Newham': 'NE - North East', 'Waltham Forest': 'NE - North East', 
                'Haringey': 'NA - North Area', 'Enfield': 'NA - North Area', 'Barnet': 'NW - North West',
                'Harrow': 'NW - North West', 'Hillingdon': 'WA - West Area'}

## Cleaning
note: no nan values removed as other parts of the row might be useful for later analysis

In [ ]:
# drops duplicate rows and useless columns

street_clean = df_metro_street.drop_duplicates().drop(columns=['Context', 'Unnamed: 0']).copy()

In [ ]:
# map borough / bcu to dataframe

street_clean['Borough'] = street_clean['LSOA name'].str[:-5]
street_clean = street_clean[street_clean['Borough'].isin(boroughs_bcu.keys())]
street_clean['BCU'] = street_clean['Borough'].map(boroughs_bcu)

In [ ]:
# drops duplicate rows

outcomes_clean = df_metro_outcomes.drop_duplicates().copy()

In [ ]:
# map borough / bcu to dataframe

outcomes_clean['Borough'] = outcomes_clean['LSOA name'].str[:-5]
outcomes_clean = outcomes_clean[outcomes_clean['Borough'].isin(boroughs_bcu.keys())]
outcomes_clean['BCU'] = outcomes_clean['Borough'].map(boroughs_bcu)

In [ ]:
# drops useless columns

search_clean = df_metro_search.drop(columns=['Removal of more than just outer clothing', 'Outcome linked to object of search', 
                                             'Policing operation', 'Part of a policing operation', 'Unnamed: 0']).copy()

In [ ]:
# correct time format

search_clean['Date'] = search_clean['Date'].str[:7]

In [ ]:
# calculates max / min lat, long values in cleaned tables to apply to search data (no lsoa available for search data)

max_lat = max([outcomes_clean['Latitude'].max(), street_clean['Latitude'].max()])
min_lat = min([outcomes_clean['Latitude'].min(), street_clean['Latitude'].min()])
max_long = max([outcomes_clean['Longitude'].max(), street_clean['Longitude'].max()])
min_long = min([outcomes_clean['Longitude'].min(), street_clean['Longitude'].min()])

In [ ]:
# removes all rows for which the lat / long doesn't fall in the max min values of other tables 

search_clean = search_clean[(search_clean['Latitude'] <= max_lat) & (search_clean['Latitude'] >= min_lat)]
search_clean = search_clean[(search_clean['Longitude'] <= max_long) & (search_clean['Longitude'] >= min_long)]

## Adding back to database

In [ ]:
street_clean.to_sql('METRO_STREET', conn_metro, if_exists="replace", index=False)

In [ ]:
outcomes_clean.to_sql('METRO_OUTCOMES', conn_metro, if_exists="replace", index=False)

In [ ]:
search_clean.to_sql('METRO_SEARCH', conn_metro, if_exists="replace", index=False)

In [8]:
conn_metro.close()